In [13]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import scipy.stats as st
import analisis_tecnico as at
from sklearn import linear_model
import statsmodels.stats.api as sms

In [14]:
def intervalos(list):
    inter = st.bayes_mvs(list, 0.95)
    col = ['Estadistico', 'minimo', 'maximo']
    ind = ['Media', 'Varianza', 'Desviacion']
    conf = [[inter[0][0], inter[0][1][0], inter[0][1][1]], [inter[1][0], inter[1][1][0], inter[1][1][1]],
            [inter[2][0], inter[2][1][0], inter[2][1][1]]]
    ic = pd.DataFrame(conf, columns=col, index=ind)
    return ic

In [15]:
repl_list = {',': '.', '%': '', 'K': '000','-': 10000}

df = pd.read_csv('copper_daily.csv', sep=',')
df.replace(repl_list, regex=True, inplace=True)

fecha = np.array(df['Date'])[::-1]
apertura = np.array(df['Open'])[::-1]
maximo = np.array(df['High'])[::-1]
minimo = np.array(df['Low'])[::-1]
cierre = np.array(df['Price'])[::-1]
variacion = cierre - apertura

maximo_global = np.amax(maximo)
minimo_global = np.amin(minimo)

var = (maximo_global - minimo_global) / 100
sesiones = 5

aperturas_clave_reducidas = at.aperturas_clave_reducidas(apertura, sesiones, var)
media_movil1=at.media_movil(apertura, 20)
media_movil2=at.media_movil(apertura, 200)

soportes_resistencias = np.zeros((len(aperturas_clave_reducidas), len(apertura)))

for i in range(len(aperturas_clave_reducidas)):
    for k in range(len(apertura)):
        soportes_resistencias[i, k] = aperturas_clave_reducidas[i]

In [16]:
print(df.head(), '\n --------------\n', df.describe())

sms.DescrStatsW(df['Open']).tconfint_mean()


           Date  Price   Open   High    Low   Vol. Change %
0  May 05. 2020  2.335  2.325  2.340  2.323  10000     0.65
1  May 04. 2020  2.320  2.297  2.323  2.284  10000     1.00
2  May 03. 2020  2.297  2.312  2.312  2.291  10000    10000
3  May 01. 2020  2.311  2.332  2.333  2.286  10000    10000
4  Apr 30. 2020  2.333  2.377  2.394  2.319  10000    10000 
 --------------
              Price         Open         High          Low
count  1373.000000  1373.000000  1373.000000  1373.000000
mean      2.662304     2.630935     2.647234     2.614315
std       0.336055     0.322892     0.322047     0.323895
min       1.993000     1.937000     1.962000     1.937000
25%       2.383000     2.369000     2.393000     2.353000
50%       2.674000     2.660000     2.679000     2.643000
75%       2.843000     2.845000     2.858000     2.822000
max       3.404000     3.370000     3.370000     3.370000


(2.613840798684275, 2.6480295581984645)

In [17]:
fig = go.Figure(data=[go.Candlestick(x=df['Date'],open = df['Open'], high = df['High'], 
                                     low = df['Low'], close = df['Price'], name = 'Velas')])
fig.update_layout(xaxis_rangeslider_visible=False)
fig.add_scatter(x=df['Date'], y=df['Price'], mode='lines',line_color = 'blue', name = 'Cierre')
fig.add_scatter(x=df['Date'], y=media_movil1, mode='lines',line_color = 'green', name = 'Media movil 1')
fig.add_scatter(x=df['Date'], y=media_movil1, mode='lines',line_color = 'gray', name = 'Media movil 2')
for i in range(len(aperturas_clave_reducidas)):
    fig.add_scatter(x=df['Date'], y=soportes_resistencias[i], mode='lines',line_color = 'yellow',showlegend = False)

fig.show()

In [18]:
porc = int(len(df)/3)
train = df.loc[0:porc][['Price','Open','High','Low','Vol.']]
test = df.loc[porc+1:len(df)]

train_x = train.loc[0:porc-1]
train_y = test.loc[1:porc]

In [19]:
train

,Price,Open,High,Low,Vol.
0,2.335,2.325,2.340,2.323,10000
1,2.320,2.297,2.323,2.284,10000
2,2.297,2.312,2.312,2.291,10000
3,2.311,2.332,2.333,2.286,10000
4,2.333,2.377,2.394,2.319,10000
...,...,...,...,...,...
453,2.809,2.749,2.793,2.749,0.01000
454,2.770,2.773,2.773,2.760,0.05000
455,2.784,2.780,2.780,2.780,0.03000
456,2.796,2.789,2.789,2.780,0.01000


In [20]:
float(df['Change %'][4])

10000.0

In [21]:
df['Change %'] = [float(df['Change %'][i]) for i in range(len(df))]

In [22]:
df.describe()

,Price,Open,High,Low,Change %
count,1373.000000,1373.000000,1373.000000,1373.000000,1373.000000
mean,2.662304,2.630935,2.647234,2.614315,4938.529111
std,0.336055,0.322892,0.322047,0.323895,5001.006325
min,1.993000,1.937000,1.962000,1.937000,0.000000
25%,2.383000,2.369000,2.393000,2.353000,0.580000
50%,2.674000,2.660000,2.679000,2.643000,3.560000
75%,2.843000,2.845000,2.858000,2.822000,10000.000000
max,3.404000,3.370000,3.370000,3.370000,10000.000000


float